# Part 3: EM implementation

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [212]:
def read_data():
    with open('sequence.padded.txt') as handle:
        # I dont think this is the actual sequences we should be using
        # put using as placeholder for now.
        return [s.strip() for s in handle.readlines()]

data = read_data()
data

['ATACCCCTGGCTGGGTCATGGTGACCTGGAGGAAGCGT',
 'CATATATGGCCAGGGTCAGTGTGACCTCCATTTCCCAT',
 'AGCAGCTGGCCTGGGTCACAGTGACCTGACCTCAAACC',
 'AGGCTGTGTACAAGGTCAGAGTGACCTCTAGAAGCTCT',
 'TACTCTAGTTCCAGGTCATGGTGACCTGTGAAAAATCT',
 'AGGACTGTTTCAAGGTCACGGTGACCCTCGTGGGCTGT',
 'GCAGGAAGTTTTGGGTCACGGTGACCTCTAGTTGTTGA',
 'CAAGTGCTTCAAAGGTCATGGTGCCCTGGGGCCGAGAG',
 'ACCAACATGGCAGGGTCAAGTTGACCTCCCTGGCCACT',
 'TCTCTCTCTAGTAGGTCATGGTGACCTGTACACATTAT',
 'TCAGACCACAGAGGGTCAAGGTGACCTGAGAGATCAGT',
 'AGGCAATTCACTAGGTCAGGATGCCCTGGGGCAACAGT',
 'TAGTCCTGAAAAGGGTCATGTTGACCTGATTGTCATGT',
 'ATTAACTCTTCTAGGTCAGTGTGACCTAAACTCATCGG',
 'GGACAATTATTGGGGTCACGGTGACCTGCCTGTTTCAG',
 'GGTCCATAATATAGGTCATGTTGACCTGGGACAACTGG',
 'CTCCAGGAGCAGGGGTCAGGGTGACCTCCAGCTCCTCA',
 'GAGCCCATCTCTGGGTCATGTTGCCCTCTTACAGCACA',
 'TGGGTTAAACCTGGGTCATGTTGACCTAGATACATCTC',
 'GTGACATCCCCAGGGTCAAAGTGCCCTGAGTCTGGAGA',
 'GCCTTCTAGGTCAGCATGACCTGGTCCTCAGAGGGGGG',
 'GGCAATGAATCAAGGTCAGGCTAACCTGGCTTACTGCA',
 'CCTACTAGCCCTGGGTCAACGTGCCCTGTAAGAGCATG',
 'GGCGCAGCC

Create matrix $X_{i,j,p,k}$

In [187]:
seq_length = len(data[0])
motif_length = 8
number_motifs = seq_length - motif_length + 1
X = np.zeros((len(data), seq_length, motif_length, 4))

def nuc_to_one_hot(nuc):
    # Convert nucleotide to the index in one hot encoded array
    # that should be hot (==1)
    upper_nuc = nuc.upper()
    mapping = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    return mapping[upper_nuc]

j_p = []
for i in range(seq_length):
    for j in range(number_motifs):
        for p in range(motif_length):
            nuc = data[i][j+p]
            k_hot = nuc_to_one_hot(nuc)
            X[i][j][p][k_hot] = 1.0

assert X.sum() == seq_length * number_motifs * motif_length

Initialize model parameters.

In [163]:
def init_EM(seq_length, motif_length):
    number_motifs_per_sequence = seq_length - motif_length + 1
    lambda_j = np.random.uniform(0, 1, size=(number_motifs_per_sequence,))
    lambda_j_norm = lambda_j / lambda_j.sum()
    psi_0 = np.random.uniform(0, 1, size=(4, motif_length))
    psi_1 = np.random.uniform(0, 1, size=(4, motif_length))
    psi_0 = (psi_0.T/psi_0.sum(axis=1)).T  # https://stackoverflow.com/questions/16202348/numpy-divide-row-by-row-sum
    psi_1 = (psi_1.T/psi_1.sum(axis=1)).T
    
    return lambda_j_norm, psi_0, psi_1

In [60]:
lambda_j, psi_0, psi_1 = init_EM(seq_length, motif_length)

psi[1]

array([0.296616  , 0.19186784, 0.05512813, 0.04871015, 0.08853719,
       0.17004914, 0.1211089 , 0.02798265])

## E step numerator

- $\prod_{p}\prod_{k} {\psi^{1}}$ is product of probabilities of the nucleotides in a given motif J 
- $\prod_{j'!=j}\prod_{p}\prod_{k}$ is product of probabilities of the nucleotides of all motifs for the same sequence that are not motif J.

The numerator is then the sum of everything.

### Questions
- What does the product of $P(C_{i} = j | X, \theta)$ look like in terms of shape?

If asking what is the prob of $C_{i} = j$ then you are really asking what is the prob of motif at position j being the transcription factor binding site. You need to be able to answer this question for all $j$. So you should have an array with shape `(number of sequences, number of motifs per sequence`).

- How does taking the log here work? Products become sums so below may not actually be correct. I don't think we can just take the log whenever we want. I am not sure I understand Quon description of how to work in log space.

In [ ]:
def E_step(X, lambda_j, psi_0, psi_1):
    num = lambda_j * (X * psi_1.T) * (X * psi_1.T)
    denom = lambda_j * (X * psi_1.T) * (X * psi_1.T)

First take product along axis 3. This will just return an array of what were the non-zero nucleotide probibilties ($\prod_{k}$).

In [190]:
prod = X * psi_1.T
prod_3 = prod.prod(axis=3)
prod_3.shape

(357, 38, 8)

In [192]:
prod[0][0]  # one motif

array([[0.1752402 , 0.        , 0.        , 0.        ],
       [0.        , 0.13732127, 0.        , 0.        ],
       [0.04151508, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.16145532],
       [0.        , 0.        , 0.        , 0.14845187],
       [0.        , 0.        , 0.        , 0.06785852],
       [0.        , 0.        , 0.        , 0.10579959],
       [0.        , 0.15758928, 0.        , 0.        ]])

Then take the product along axis 2 of this new array. This is the product for each motif ($\prod_{p}$)

In [193]:
prod_3.prod(axis=2).shape

(357, 38)

Combine into one operation.

In [202]:
prod_pk_psi_1 = (X * psi_1.T)
prod_pk_psi_1[prod_pk_psi_1 == 0] = 1

prod_pk_psi_1 = np.log(prod_pk_psi_1)
prod_pk_psi_1[prod_pk_psi_1 == 0] = 1

n1 = prod_pk_psi_1.prod(axis=3).prod(axis=2)
n1.shape

(357, 38)

In [211]:
prod_pk_psi_0 = (X * psi_0.T)
prod_pk_psi_0[prod_pk_psi_0 == 0] = 1

prod_pk_psi_0 = np.log(prod_pk_psi_0)
prod_pk_psi_0[prod_pk_psi_0 == 0] = 1
n2 = prod_pk_psi_0.prod(axis=3).prod(axis=2)
n2.shape

(357, 38)

In [206]:
prod_pk_psi_0

array([[[[-1.68987363,  1.        ,  1.        ,  1.        ],
         [ 1.        , -3.27685747,  1.        ,  1.        ],
         [-2.80740439,  1.        ,  1.        ,  1.        ],
         ...,
         [ 1.        ,  1.        ,  1.        , -1.48378766],
         [ 1.        ,  1.        ,  1.        , -2.86825398],
         [ 1.        , -1.29040552,  1.        ,  1.        ]],

        [[ 1.        , -2.98937012,  1.        ,  1.        ],
         [-2.2188133 ,  1.        ,  1.        ,  1.        ],
         [ 1.        ,  1.        ,  1.        , -1.27827188],
         ...,
         [ 1.        ,  1.        ,  1.        , -1.48378766],
         [ 1.        , -2.25069662,  1.        ,  1.        ],
         [ 1.        ,  1.        , -1.65146286,  1.        ]],

        [[-1.68987363,  1.        ,  1.        ,  1.        ],
         [ 1.        ,  1.        ,  1.        , -3.09299619],
         [ 1.        ,  1.        ,  1.        , -1.27827188],
         ...,
         

## M step

Quon says $\boldsymbol{E}[C_{i,j}] = P(C_{i} = j | X_{i}, \theta)$

He also gives what $\boldsymbol{E}[C_{i,j}] = P(C_{i} = j | X_{i}, \theta)$ equals to in the E step (shown in the image below.)

![](e.png)

### $\lambda_{j}$

Take sum of all values at each index $i$ over vector $C_{i, j}$ which would store prob each $j$ (each motif) being the transcription factor binding site and divide this value by the number of sequences $N$.
- Would this not always just sum to 1? And therefore $\lambda_{j}$ is basically fixed at 1 over the number of sequences?

### $\psi^{1}_{p, k}$

- How does taking a sum over $i$ for $C_{i, j}$look compared to taking a sum over $i$ and $j$?

Product of indicator variables for a given motif (For example the matrix at `X[0][0]`) and the expectation at that motif calculated during the E step. Then take a sum overall all those values and divide by the number of sequences.

### $\psi^{0}_{p, k}$

Seems pretty much like other $\psi$ but add some subtractions and change the denominator to the number of sequences times the number of possible motifs.